In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

2023-07-10 01:55:25.455680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Define the SNN layer

In [17]:
class SpikingLayer(layers.Layer):
    def __init__(self, num_neurons, **kwargs):
        super(SpikingLayer, self).__init__(**kwargs)
        self.num_neurons = num_neurons

    def build(self, input_shape):
        self.threshold = self.add_weight(
            shape=(input_shape[-1],),
            initializer="zeros",
            trainable=False,
            name="threshold"
        )

    def call(self, inputs):
        spiking_output = tf.nn.relu(inputs - self.threshold)
        self.threshold.assign_add(tf.reduce_mean(spiking_output, axis=0))
        return spiking_output

In [21]:
model = tf.keras.Sequential([
    layers.Input(shape=(784,)),
    SpikingLayer(num_neurons=64),
    SpikingLayer(num_neurons=128),
    SpikingLayer(num_neurons=256),
    layers.Dense(
        10, 
        activation="softmax",
        name = "output"
    )
])

In [23]:
adam = tf.keras.optimizers.Adam(
    learning_rate = 0.001
)

model.compile(
    optimizer="adam", 
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
    metrics=["accuracy"]
)

model.fit(
    x_train, 
    y_train, 
    batch_size=10240, 
    epochs=20, 
    validation_data=(x_test, y_test),
    verbose = 1
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/20
6/6 [==============================] - 1s 67ms/step - loss: 2.3017 - accuracy: 0.1069 - val_loss: 2.3018 - val_accuracy: 0.1042
Epoch 2/20
6/6 [==============================] - 0s 41ms/step - loss: 2.3014 - accuracy: 0.1078 - val_loss: 2.3016 - val_accuracy: 0.1043
Epoch 3/20
6/6 [==============================] - 0s 41ms/step - loss: 2.3011 - accuracy: 0.1083 - val_loss: 2.3014 - val_accuracy: 0.1045
Epoch 4/20
6/6 [==============================] - 0s 41ms/step - loss: 2.3009 - accuracy: 0.1136 - val_loss: 2.3012 - val_accuracy: 0.1227
Epoch 5/20
6/6 [==============================] - 0s 41ms/step - loss: 2.3007 - accuracy: 0.1264 - val_loss: 2.3010 - val_accuracy: 0.1223
Epoch 6/20
6/6 [==============================] - 0s 42ms/step - loss: 2.3005 - accuracy: 0.1261 - val_loss: 2.3008 - val_accuracy: 0.1221
Epoch 7/20
6/6 [==============================] - 0s 41ms/step - loss: 2.3003 - accuracy: 0.1260 - val_loss: 2.3007 - val_accuracy: 0.1222
Epoch 8/20
6/6 [===========

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spiking_layer_7 (SpikingLay  (None, 784)              784       
 er)                                                             
                                                                 
 spiking_layer_8 (SpikingLay  (None, 784)              784       
 er)                                                             
                                                                 
 spiking_layer_9 (SpikingLay  (None, 784)              784       
 er)                                                             
                                                                 
 output (Dense)              (None, 10)                7850      
                                                                 
Total params: 10,202
Trainable params: 7,850
Non-trainable params: 2,352
_______________________________________________

In [7]:
model.save("SNN_MNIST.h5")

# hls4ml Config Part

In [8]:
from tensorflow.keras.models import load_model

model = load_model("SNN_MNIST.h5", custom_objects={'SpikingLayer': SpikingLayer})

In [9]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)
model.summary()

313/313 [==============================] - 0s 769us/step - loss: 2.3006 - accuracy: 0.1051
Test Loss: 2.300551176071167
Test Accuracy: 0.10509999841451645
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spiking_layer (SpikingLayer  (None, 784)              784       
 )                                                               
                                                                 
 output (Dense)              (None, 10)                7850      
                                                                 
Total params: 8,634
Trainable params: 7,850
Non-trainable params: 784
_________________________________________________________________


In [10]:
import hls4ml

class HSpikingLayer(hls4ml.model.layers.Layer):
    '''
    def __init__(self, name, attributes, inputs, outputs):
        super().__init__()
    '''
    def initialize(self):
        inp = self.get_input_variable()
        shape = inp.shape
        dims = inp.dim_names
        self.add_output_variable(shape, dims)
        self.threshold = self.create_local_variable(shape=(shape[-1],), dtype=inp.dtype, name='threshold')

    def function_cpp(self):
        input_var = self.get_input_variable_cpp()
        output_var = self.get_output_variable_cpp()

        cpp_code = f'''
        // Calculate spiking output
        {output_var} = relu({input_var} - {self.threshold});
        // Update threshold
        {self.threshold} += hls4ml::math::reduce_mean({output_var}, {output_var}.size());
        '''
        return cpp_code

In [11]:
def parse_spiking_layer(keras_layer, input_names, input_shapes, data_reader):
    layer = {}
    layer['class_name'] = 'HSpikingLayer'  # 自定义层的hls4ml层名称
    layer['name'] = keras_layer['config']['name']
    layer['num_neurons'] = keras_layer['config']['num_neurons']

    if input_names is not None:
        layer['inputs'] = input_names

    return layer, input_shapes

In [12]:
def register_custom_layer():
    hls4ml.converters.register_keras_layer_handler('SpikingLayer', parse_spiking_layer)
    hls4ml.model.layers.register_layer('HSpikingLayer', HSpikingLayer)

In [13]:
register_custom_layer()

In [14]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
#config['InputShape'] = {'spiking_layer_input': (None, 784)}
config['Model']['Precision'] = 'ap_fixed<12,6>'
config['Model']['ReuseFactor'] = 1
'''
for Layer in config['LayerName'].keys():
    config['LayerName'][Layer]['Strategy'] = 'Latency'
    config['LayerName'][Layer]['ReuseFactor'] = 1
    #config['LayerName'][Layer]['Precision'] = 'ap_fixed<8,4>'
'''
config['LayerName']['output']['Strategy'] = 'Stable'
'''
for layer in ['conv1', 'conv2'] :
    config['LayerName'][layer]['Precision'] = 'ap_fixed<8,4>'
'''
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

cfg = hls4ml.converters.create_config(backend='VivadoAccelerator')
cfg['IOType'] = 'io_stream'
cfg['HLSConfig'] = config
cfg['KerasModel'] = model
cfg['OutputDir'] = 'AlexNet_PYNQ'
cfg['Board'] = 'pynq-z2'

hls_model = hls4ml.converters.keras_to_hls(cfg)

hls_model.compile()

Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 784]], output shape: [None, 784]
Layer name: spiking_layer, layer type: HSpikingLayer, input shapes: [[None, 784]], output shape: [[None, 784]]
Layer name: output, layer type: Dense, input shapes: [[[None, 784]]], output shape: [10]
-----------------------------------
Model
  Precision:         ap_fixed<12,6>
  ReuseFactor:       1
  Strategy:          Latency
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  input_1
    Trace:           False
    Precision
      result:        fixed<16,6>
  spiking_layer
    Trace:           False
    Precision
      result:        fixed<16,6>
  output
    Trace:           False
    Precision
      result:        fixed<16,6>
      weight:        fixed<16,6>
      bias:          fixed<16,6>
    Strategy:        Stable
  output_softmax
    Trace:           False
    Precision
      result:        fixed<16,6>
-------------------------

AttributeError: 'VivadoAcceleratorHSpikingLayer' object has no attribute 'create_local_variable'

In [ ]:
import os

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']
os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libudev.so.1'

In [ ]:
hls_model.build(csim=False, export=True, bitfile=True)

In [ ]:
!sed -n '30,45p' AlexNet_Alveo50/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_utilization_placed.rpt